# Motif Discovery Visualization

## Imports

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
from pathlib import Path
from ipywidgets import interact_manual
from scipy import stats

from scripts.runBenchmark import getBenchmarkFiles, algorithms
from scripts.plot import plotMotif

## Find benchmark files

You can generate the benchmark using the _generateBenchmark.py_ script. The benchmark path can be modified below.

In [2]:
benchmarkPath = Path("./real")  # change path to benchmark here

exists = benchmarkPath.exists()
print("Benchmark Path exists:", exists)
filePaths = {}
count = 0
if exists:
    for files in getBenchmarkFiles(benchmarkPath):
        filePaths[files[0]] = files[1:]
        count += 1
print(count, "time series found")

Benchmark Path exists: True
6 time series found


## Run an algorithm and plot results

In [4]:
timeout = None#10.0  # set the timeout (in seconds) here
maxMemory = None#6144*1024*1024  # set memory limit (in B) here

@interact_manual(Algorithm=algorithms.keys(), TS_Files=sorted(filePaths.keys()))
def interactive_plot(Algorithm, TS_Files):
    tsDir = Path(TS_Files)
    tsPath = tsDir / filePaths[TS_Files][0]
    tsMetaPath = tsDir / filePaths[TS_Files][1]
    motif,ts,l = plotMotif(Algorithm, tsPath, tsMetaPath, timeout, maxMemory)

    print(len(motif))
    print(motif)
    print([i%48 for i in motif])
    fig, ax = plt.subplots(figsize=(5,5))
    yOffset = 0
    for i in motif[:5]:
        y = stats.zscore(ts[i:i+l])-yOffset
        ax.plot(y)
        yOffset+=3
    ax.set_yticks([])
    
    mean = stats.zscore(ts[motif[0]:motif[0]+l])
    for i in motif[1:]:
        mean += stats.zscore(ts[i:i+l])
    mean /= len(motif)
    fig, ax = plt.subplots(figsize=(5,5))
    ax.plot(mean)

interactive(children=(Dropdown(description='Algorithm', options=('CliqueMotif', 'EMMA', 'GrammarViz3.0', 'Scan…

You can use the buttons below the figure to interactively zoom and move the plot.

The area at the specific indices is colored as follows:
* <span style="color:green">Green</span>: true positive
* <span style="color:olive">Yellow</span>: false negative
* <span style="color:red">Red</span>: false positive
* White: true negative

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

In [3]:
from cliqueMotif import getTopMotif
import math
import numpy as np

timeout = None#10.0  # set the timeout (in seconds) here
maxMemory = None#6144*1024*1024  # set memory limit (in B) here

@interact_manual(Algorithm=algorithms.keys(), TS_Files=sorted(filePaths.keys()))
def interactive_plot(Algorithm, TS_Files):
    tsDir = Path(TS_Files)
    tsPath = tsDir / filePaths[TS_Files][0]
    tsMetaPath = tsDir / filePaths[TS_Files][1]
    l = 48
    minCorr = 0.85
    runs = 30
    maxRange = math.sqrt(l/2*(1-minCorr))
    corrs = []
    ranges = []
    lens = []
    for r in np.linspace(0,maxRange,runs):
        corrs.append(1-2*(r**2/l))
        ranges.append(r)
        motif,stats = getTopMotif(l, r, tsPath)
        print(r,",",len(motif))
        lens.append(len(motif))
    fig, ax = plt.subplots()
    ax.plot(corrs,lens)

interactive(children=(Dropdown(description='Algorithm', options=('CliqueMotif', 'EMMA', 'GrammarViz3.0', 'Scan…